In [1]:
import urllib.request
import urllib.parse
import json
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm 

### 네이버 검색 API로 블로그 검색

In [3]:
import urllib.parse
import urllib.request
import json

# access information
client_id = "Bs4d81MbFjUn7Mfc2oO5"
client_secret = "NvF9NY3CPs"

# 키워드 리스트
keywords = ["노트북 리뷰", "노트북 추천", "태블릿 후기", "태블릿 추천"]

# 결과 저장
all_results = []

for keyword in keywords:
    # 키워드 인코딩
    encText = urllib.parse.quote(keyword)
    url = f"https://openapi.naver.com/v1/search/blog?query={encText}&display=10&sort=sim"

    # API 요청
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)

    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if rescode == 200:
        response_body = response.read()
        data = json.loads(response_body.decode('utf-8'))['items']
        all_results.extend(data)  # 결과 저장
        print(f"키워드 '{keyword}'에 대한 데이터를 수집했습니다.")
    else:
        print(f"Error Code: {rescode}")

# 결과 확인
print(f"총 {len(all_results)}개의 데이터를 수집했습니다.")


키워드 '노트북 리뷰'에 대한 데이터를 수집했습니다.
키워드 '노트북 추천'에 대한 데이터를 수집했습니다.
키워드 '태블릿 후기'에 대한 데이터를 수집했습니다.
키워드 '태블릿 추천'에 대한 데이터를 수집했습니다.
총 40개의 데이터를 수집했습니다.


### 네이버 블로그 크롤링
- 정규표현식으로 HTML에서 필요한 내용만 추출

In [4]:
# 크롬 드라이버 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 브라우저 창을 띄우지 않음
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

chrome_driver_path = "C:/Users/highk/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe"  # 크롬 드라이버 경로 설정
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

In [5]:
import re

# 정규 표현식을 이용하여 img 태그에서 data-lazy-src 속성만 남기고, 나머지 HTML 태그는 삭제
img_pattern = r'<img\b[^>]*\bdata-lazy-src="([^"]+)"[^>]*>'
img_replace = r'<img data-lazy-src="postfiles.pstatic.net" />'

# 네이버 지도 이미지를 모두 삭제 -> 직접 첨부한 이미지가 아니면 모두 제거
# map_pattern = r'<img\b[^>]*\bsrc="[^"]*map\.pstatic[^"]*"[^>]*>'
non_postfiles_pattern = r'<img\b[^>]*\bsrc="(?![^"]*postfiles\.pstatic\.net)[^"]*"[^>]*>'

# 불필요한 다른 HTML 태그를 삭제 (# img 태그를 제외한 모든 태그 삭제)
html_pattern = r'<(?!img\b)[^>]+>'
flash_pattern = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""

# 동영상 제거
video_div_pattern = r'<div\b[^>]*\bclass="[^"]*se-video[^"]*"[^>]*>[\s\S]*?</div>'

# 연속된 \n을 단일 \n로 치환
newline_pattern = r'\n+'
tab_pattern = r'\t+'

In [6]:
def scrape_blog_selenium(url):
    driver.get(url)
    time.sleep(4)
    
    iframe = driver.find_element(By.ID , "mainFrame") 
    driver.switch_to.frame(iframe) 

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    content = soup.select("div.se-main-container")
    content = ''.join(str(content))
    
    # remove video tags
    formatted_content = re.sub(pattern=video_div_pattern, repl='', string=content, flags=re.DOTALL) # DOTALL 플래그로 여러 줄에 걸친 내용을 매칭
    # remove img tags without postfiles.pstatic.net url
    formatted_content = re.sub(pattern=non_postfiles_pattern, repl='', string=formatted_content)
    # remove unnecessary properties of img tag
    formatted_content = re.sub(pattern=img_pattern, repl=img_replace, string=formatted_content)
    
    # remove html tags without img tag
    formatted_content = re.sub(pattern=html_pattern, repl='', string=formatted_content)
    # remove flash callback function
    formatted_content = formatted_content.replace(flash_pattern, '')
    
    # strip and organize \n
    formatted_content = re.sub(pattern=newline_pattern, repl='\n', string=formatted_content)
    formatted_content = re.sub(pattern=tab_pattern, repl='', string=formatted_content)
    formatted_content = formatted_content.replace('\u200b', '')

    return [content, formatted_content]


# 각 블로그 URL을 순회하며 데이터 스크래핑
blogs_data = []
for blog in tqdm(data, desc="Scraping Blogs", unit="blog"):
    content, formatted_content = scrape_blog_selenium(blog['link'])
    blogs_data.append({ "title": blog['title'], "content": formatted_content, "raw_content": content, "link": blog['link'] })

Scraping Blogs:   0%|          | 0/10 [00:00<?, ?blog/s]

Scraping Blogs: 100%|██████████| 10/10 [00:59<00:00,  5.98s/blog]


In [7]:
blogs_data[0]

{'title': '가성비 <b>태블릿 추천</b> 레노버 리전 Y700 2세대 실사용기',
 'content': '[\n가성비 태블릿 추천 레노버 리전 Y700 2세대 실사용기\n필자가 레노버 Y700 2세대를 사용한 지도 어느덧 6개월이란 시간이 흘렀습니다. 전체적인 사용감을 고려해 봤을 때, 만족스러운 포인트가 많았던 모델인데요. 가성비 태블릿 추천 이유와 함께 그 후기를 가볍게 남겨보려고 합니다 :)\n \n목차1. 준수한 디자인, 그리고 휴대성2. 디스플레이3. 성능4. 맺음\n \n준수한 디자인,그리고 휴대성\n01\n<img data-lazy-src="postfiles.pstatic.net" />\n<img data-lazy-src="postfiles.pstatic.net" />\n \n사실 필자의 관점에서 중국 브랜드의 태블릿이라 하면 디자인이 불호인 경우가 많았습니다. 물론, 리전 Y700 2세대의 외관도 특별하다 할 정도는 아니지만, 준수한 마감이라던가 후면에 새겨진 LEGION 로고나 카메라 부분은 디자인적으로 멋지다는 인상을 남겨주기에 충분했습니다.\n \n<img data-lazy-src="postfiles.pstatic.net" />\n웹 브라우저에 최적화된 세로 모드\n화면 크기도 8.8인치인데다 무게도 약 350g 정도에 불과한 수준이다 보니 휴대성에 특화되었다는 특징이 있습니다. 한 손으로 들기에 부담이 없을뿐더러, 지하철이나 버스에서 출퇴근하면서 이용하기에도 용이한 편입니다.\n \n디스플레이\n02\n<img data-lazy-src="postfiles.pstatic.net" />\n레노버 Y700 2세대는 8.8인치(22.35cm) 크기에 2.5K 해상도(2560x1600), 16:10 화면 비율, 최대 144Hz의 고주사율과 500니트의 밝기를 지원하는 디스플레이를 탑재하고 있습니다. 광량이 높은 외부에서도 편안하게 화면을 바라볼 수도 있지만, 높은 주사율까지 지원하면서 부드러운 화면 전환과 애니메이션 효과를 즐길 수 있습

In [8]:
# 결과 출력
for idx, blog in enumerate(blogs_data):
    print(f"블로그 {idx + 1}")
    print(f"제목: {blog['title']}")
    print(f"본문: {blog['content']}...")  # 본문 일부 출력
    # print(f"이미지 개수: {len(blog['images'])}")
    # print(f"이미지 URL: {blog['images']}")
    print("-" * 80)

블로그 1
제목: 가성비 <b>태블릿 추천</b> 레노버 리전 Y700 2세대 실사용기
본문: [
가성비 태블릿 추천 레노버 리전 Y700 2세대 실사용기
필자가 레노버 Y700 2세대를 사용한 지도 어느덧 6개월이란 시간이 흘렀습니다. 전체적인 사용감을 고려해 봤을 때, 만족스러운 포인트가 많았던 모델인데요. 가성비 태블릿 추천 이유와 함께 그 후기를 가볍게 남겨보려고 합니다 :)
 
목차1. 준수한 디자인, 그리고 휴대성2. 디스플레이3. 성능4. 맺음
 
준수한 디자인,그리고 휴대성
01
<img data-lazy-src="postfiles.pstatic.net" />
<img data-lazy-src="postfiles.pstatic.net" />
 
사실 필자의 관점에서 중국 브랜드의 태블릿이라 하면 디자인이 불호인 경우가 많았습니다. 물론, 리전 Y700 2세대의 외관도 특별하다 할 정도는 아니지만, 준수한 마감이라던가 후면에 새겨진 LEGION 로고나 카메라 부분은 디자인적으로 멋지다는 인상을 남겨주기에 충분했습니다.
 
<img data-lazy-src="postfiles.pstatic.net" />
웹 브라우저에 최적화된 세로 모드
화면 크기도 8.8인치인데다 무게도 약 350g 정도에 불과한 수준이다 보니 휴대성에 특화되었다는 특징이 있습니다. 한 손으로 들기에 부담이 없을뿐더러, 지하철이나 버스에서 출퇴근하면서 이용하기에도 용이한 편입니다.
 
디스플레이
02
<img data-lazy-src="postfiles.pstatic.net" />
레노버 Y700 2세대는 8.8인치(22.35cm) 크기에 2.5K 해상도(2560x1600), 16:10 화면 비율, 최대 144Hz의 고주사율과 500니트의 밝기를 지원하는 디스플레이를 탑재하고 있습니다. 광량이 높은 외부에서도 편안하게 화면을 바라볼 수도 있지만, 높은 주사율까지 지원하면서 부드러운 화면 전환과 애니메이션 효과를 즐길 수 있습니다.
 
<img data-lazy-src="postfi

In [9]:
blogs_data

[{'title': '가성비 <b>태블릿 추천</b> 레노버 리전 Y700 2세대 실사용기',
  'content': '[\n가성비 태블릿 추천 레노버 리전 Y700 2세대 실사용기\n필자가 레노버 Y700 2세대를 사용한 지도 어느덧 6개월이란 시간이 흘렀습니다. 전체적인 사용감을 고려해 봤을 때, 만족스러운 포인트가 많았던 모델인데요. 가성비 태블릿 추천 이유와 함께 그 후기를 가볍게 남겨보려고 합니다 :)\n \n목차1. 준수한 디자인, 그리고 휴대성2. 디스플레이3. 성능4. 맺음\n \n준수한 디자인,그리고 휴대성\n01\n<img data-lazy-src="postfiles.pstatic.net" />\n<img data-lazy-src="postfiles.pstatic.net" />\n \n사실 필자의 관점에서 중국 브랜드의 태블릿이라 하면 디자인이 불호인 경우가 많았습니다. 물론, 리전 Y700 2세대의 외관도 특별하다 할 정도는 아니지만, 준수한 마감이라던가 후면에 새겨진 LEGION 로고나 카메라 부분은 디자인적으로 멋지다는 인상을 남겨주기에 충분했습니다.\n \n<img data-lazy-src="postfiles.pstatic.net" />\n웹 브라우저에 최적화된 세로 모드\n화면 크기도 8.8인치인데다 무게도 약 350g 정도에 불과한 수준이다 보니 휴대성에 특화되었다는 특징이 있습니다. 한 손으로 들기에 부담이 없을뿐더러, 지하철이나 버스에서 출퇴근하면서 이용하기에도 용이한 편입니다.\n \n디스플레이\n02\n<img data-lazy-src="postfiles.pstatic.net" />\n레노버 Y700 2세대는 8.8인치(22.35cm) 크기에 2.5K 해상도(2560x1600), 16:10 화면 비율, 최대 144Hz의 고주사율과 500니트의 밝기를 지원하는 디스플레이를 탑재하고 있습니다. 광량이 높은 외부에서도 편안하게 화면을 바라볼 수도 있지만, 높은 주사율까지 지원하면서 부드러운 화면 전환과 애니메이션 효과를 즐길 수 

In [12]:
import pandas as pd

df = pd.DataFrame(blogs_data)


# 엑셀로 저장
file_path = r"C:\Users\highk\pypy50\99-1. IT 제품 관련 크롤링\blogs_data.xlsx"  # raw 문자열 사용
df.to_excel(file_path, index=False)
print(f"엑셀 파일이 '{file_path}'에 저장되었습니다.")

엑셀 파일이 'C:\Users\highk\pypy50\99-1. IT 제품 관련 크롤링\blogs_data.xlsx'에 저장되었습니다.
